In [1]:
import os
import scipy
import numpy as np
import pylab as plt
import pandas as pd
import seaborn as sns
from scipy import stats
from pandas import read_csv
import matplotlib.pyplot as plt
from matplotlib import pyplot as mp
import statsmodels.api as sm
import statsmodels.formula.api as smf
from scipy.io import savemat, loadmat
from scipy.stats import pointbiserialr
from matplotlib.patches import Circle
from sklearn import datasets, linear_model, preprocessing
from scipy.stats import probplot, pearsonr
from matplotlib.cbook import get_sample_data
from pandas.plotting import lag_plot
from pandas.plotting import autocorrelation_plot
from sklearn.preprocessing import scale 
from sklearn.metrics import mean_squared_error, r2_score
from matplotlib.offsetbox import (TextArea, DrawingArea, OffsetImage,AnnotationBbox)
# print(__doc__)

/Users/emilyhahn/anaconda2/lib/python2.7/site-packages/statsmodels/compat/pandas.py:56: FutureWarning: The pandas.core.datetools module is deprecated and will be removed in a future version. Please use the pandas.tseries module instead.
  from pandas.core import datetools


### Configure environment

In [2]:
dACC_group=[]
L_dlPFC_group=[]
R_dlPFC_group=[]
L_IFG_group=[]
R_IFG_group=[]
conflict_group=[]
adapt_group=[]
rt_group=[]

dACC_group_hc=[]
L_dlPFC_group_hc=[]
R_dlPFC_group_hc=[]
L_IFG_group_hc=[]
R_IFG_group_hc=[]
conflict_group_hc=[]
adapt_group_hc=[]
rt_group_hc=[]

dACC_group_pts=[]
L_dlPFC_group_pts=[]
R_dlPFC_group_pts=[]
L_IFG_group_pts=[]
R_IFG_group_pts=[]
conflict_group_pts=[]
adapt_group_pts=[]
rt_group_pts=[]

regions=['dACC','L_dlPFC','R_dlPFC','L_IFG','R_IFG']
variables=['rt','adapt','conflict']
raw_behav_dir='/Users/emilyhahn/projects/msit_modeling/behavior_preproc/msit_mri_behav'
preproc_behav_dir='/Users/emilyhahn/projects/msit_modeling/behavior_preproc/completed'
LSS_dir='/Users/emilyhahn/projects/msit_modeling/LSS_AVG'
LSS_estim_dir='/Users/emilyhahn/projects/msit_modeling/LSS_AVG_estim'
censor_dir='/Users/emilyhahn/projects/msit_modeling/censor_data'

df=pd.DataFrame({'group':[],'subject':[],'dACC':[],'L_IFG':[],'R_IFG':[],'L_dlPFC':[],'R_dlPFC':[],'trial':[],'cond':[],'acc':[],'rt':[],'conflict':[],'adapt':[]})
df_I=pd.DataFrame({'group':[],'subject':[],'dACC':[],'L_IFG':[],'R_IFG':[],'L_dlPFC':[],'R_dlPFC':[],'trial':[],'cond':[],'acc':[],'rt':[],'conflict':[],'adapt':[]})
df_C=pd.DataFrame({'group':[],'subject':[],'dACC':[],'L_IFG':[],'R_IFG':[],'L_dlPFC':[],'R_dlPFC':[],'trial':[],'cond':[],'acc':[],'rt':[],'conflict':[],'adapt':[]})
df_I_C=pd.DataFrame({"group":[],"subject":[],'rt':[],'dACC':[],'L_IFG':[],'R_IFG':[],'L_dlPFC':[],'R_dlPFC':[],'acc':[],'conflict':[],'adapt':[]})

### Define Subjects

In [3]:
SUBJ_LIST = ['hc001','hc002','hc003','hc005','hc006','hc010','hc011',\
             'hc012','hc014','hc015','hc017','hc019','hc021','hc028',\
             'hc031','hc032','hc033','hc034','hc036','hc038','hc042',\
             'pp001','pp002','pp003','pp004','pp005','pp006','pp010',\
             'pp011','pp012','pp013','pp015','pp016']

### Define Functions - Data Structs

In [4]:
def generate_roi_vars(roi_file,region,var):
    with open(roi_file, 'r') as file:
        for line in file:
            line=line.strip()
            line=float(line)
            if -4<line<4 and line!=0:
                region.append(line)
            else:
                region.append('NaN')
                print "%s beta exlcuded for %s" % (line,roi_file)
    return
def censor_tps(censor_file,censor_var,var):
    with open(censor_file, 'r') as file:
        for line in file:
            line=line.strip()
            censor_var.append(line)
    return

### Assemble Indiv and Group Data - SS output, ROI data
    This module generates DataFrames:
    df = ALL subjects, ALL trials
    df_I = ALL subjects, Incongruent trials
    df_C = ALL subjects, Congruent trials
    df_I_C =ALL subjects, Incongruent-Congruent trials
    df_hcs = CTRL subjects, ALL trials
    df_pts = PSYCH subjects, ALL trials
    df_hcs_I = CTRL subjects, Incongruent trials
    df_pts_I = PSYCH subjects, Incongruent trials
    df_hcs_C = CTRL subjects, Congruent trials
    df_pts_C = PSYCH subjects, Congruent trials

In [5]:
for SUBJ in SUBJ_LIST:
    rt=[]
    conflict=[]
    adapt=[]
    dACC=[]
    L_IFG=[]
    R_IFG=[]
    L_dlPFC=[]
    R_dlPFC=[]
    cond=[]
    trial=[]
    acc=[]
    censor=[]        
    #---------------------------------##
    ## Configure SS variables
    #---------------------------------##
    mat = loadmat(os.path.join(preproc_behav_dir,'%s_msit_ss_iter250.mat') % SUBJ)
    #---------------------------------##
    ss_outputs_xsmt = np.expand_dims(np.array([np.concatenate(arr) \
                                for arr in mat['XSmt']]).squeeze(),1)
    ss_xsmt = ss_outputs_xsmt.squeeze()
    rt.extend([float(i) for i in np.array(mat['RT'].squeeze())])
    #---------------------------------##
    conflict.extend([float(i[0]) for i in ss_xsmt])
    adapt.extend([float(i[1]) for i in ss_xsmt])
    acc.extend([float(i) for i in np.array(mat['Accuracy'].squeeze())])
    trial.extend([float(i) for i in np.array(mat['Trial'].squeeze())])
    cond.extend([float(i) for i in np.array(mat['Interference'].squeeze())])
    #---------------------------------##
    ## Configure ROI variables
    #---------------------------------##
    ## Load indiv ROI vars
    file_1=os.path.join(LSS_dir,'%s.dACC_LSS_avg_file.1D' % SUBJ)
    file_2=os.path.join(LSS_dir,'%s.L_IFG_LSS_avg_file.1D' % SUBJ)
    file_3=os.path.join(LSS_dir,'%s.R_IFG_LSS_avg_file.1D' % SUBJ)
    file_4=os.path.join(LSS_dir,'%s.L_dlPFC_LSS_avg_file.1D' % SUBJ)
    file_5=os.path.join(LSS_dir,'%s.R_dlPFC_LSS_avg_file.1D' % SUBJ)
    file_6=os.path.join(censor_dir,'msit.%s.msit_bsm.censor.1D' % SUBJ)
    #---------------------------------##
    ## Generate ROI variables
    generate_roi_vars(file_1,dACC,"dACC")
    generate_roi_vars(file_2,L_IFG,"L_IFG")
    generate_roi_vars(file_3,R_IFG,"R_IFG")
    generate_roi_vars(file_4,L_dlPFC,"L_dlPFC")
    generate_roi_vars(file_5,R_dlPFC,"R_dlPFC")
    censor_tps(file_6,censor,"censor")
    #---------------------------------##
    ## Determine group
    s=[]
    if 'hc' in SUBJ:
        s.append(1)
    elif 'pp' in SUBJ:
        s.append(2)
    GROUP_ARR=np.array(["%s" % s] * len(trial))
    group=[i.strip('[]') for i in GROUP_ARR]
    SUBJ_ARR=np.array(["%s" % SUBJ] * len(trial))
    #---------------------------------## 
    ## Scale + interpolate missing ROI + RT data points
    ## Subjects with more than two points to interpolate excluded
    dACC = preprocessing.scale(pd.Series(dACC).astype(float).interpolate())
    R_IFG = preprocessing.scale(pd.Series(R_IFG).astype(float).interpolate())
    L_IFG = preprocessing.scale(pd.Series(L_IFG).astype(float).interpolate())
    L_dlPFC = preprocessing.scale(pd.Series(L_dlPFC).astype(float).interpolate())
    R_dlPFC = preprocessing.scale(pd.Series(R_dlPFC).astype(float).interpolate())
    ## Cannot scale RT- causes many NaNs 
    rt=np.log(rt)
    #---------------------------------## 
    dACC_group.append(dACC)
    L_dlPFC_group.append(L_dlPFC)
    R_dlPFC_group.append(R_dlPFC)
    L_IFG_group.append(L_IFG)
    R_IFG_group.append(R_IFG)
    conflict_group.append(conflict)
    adapt_group.append(adapt)
    rt_group.append(rt)
    #---------------------------------##
    if 'pp' in SUBJ_ARR[0]:
        dACC_group_pts.append(dACC)
        L_dlPFC_group_pts.append(L_dlPFC)
        R_dlPFC_group_pts.append(R_dlPFC)
        L_IFG_group_pts.append(L_IFG)
        R_IFG_group_pts.append(R_IFG)
        conflict_group_pts.append(conflict)
        adapt_group_pts.append(adapt)
        rt_group_pts.append(rt)
    elif 'hc' in SUBJ_ARR[0]:
        dACC_group_hc.append(dACC)
        L_dlPFC_group_hc.append(L_dlPFC)
        R_dlPFC_group_hc.append(R_dlPFC)
        L_IFG_group_hc.append(L_IFG)
        R_IFG_group_hc.append(R_IFG)
        conflict_group_hc.append(conflict)
        adapt_group_hc.append(adapt)  
        rt_group_hc.append(rt)
    else:
        "REVIEW SUBJ ID: %s" % SUBJ
    ##---------------------------------##
    ## Append subj data to master DF
    ##---------------------------------##
    df1=pd.DataFrame({"group":group,"subject":SUBJ_ARR,'rt':rt,'dACC':dACC,\
                      'L_IFG':L_IFG,'R_IFG':R_IFG,'L_dlPFC':L_dlPFC,\
                      'R_dlPFC':R_dlPFC,'trial':trial,'cond':cond,'acc':acc,\
                      'conflict':conflict,'adapt':adapt})
    ##---------------------------------##
    df=df.append(df1)
    df1_C=df1[df1.cond == 0]
    df1_I=df1[df1.cond == 1]
    ##---------------------------------##
    S=group[0]
    df1_C=df1_C.mean(axis=0,numeric_only=True)
    df1_C['subject']=SUBJ
    df1_C['group']=S
    df1_I=df1_I.mean(axis=0,numeric_only=True)
    df1_I['subject']=SUBJ
    df1_I['group']=S
    ##---------------------------------##
    df_I=df_I.append(df1_I,ignore_index=True)
    df_C=df_C.append(df1_C,ignore_index=True)
    ##---------------------------------##
    df1_I_C=pd.DataFrame(data={"group":S,"subject":SUBJ,'rt':df1_I['rt']-df1_C['rt']},index=[1])
    df1_I_C={"group":S,"subject":SUBJ,\
             'rt':df1_I['rt']-df1_C['rt'],\
             'dACC':df1_I['dACC']-df1_C['dACC'],\
             'L_IFG': df1_I['L_IFG']-df1_C['L_IFG'],\
             'R_IFG': df1_I['R_IFG']-df1_C['R_IFG'],\
             'L_dlPFC': df1_I['L_dlPFC']-df1_C['L_dlPFC'],\
             'R_dlPFC': df1_I['R_dlPFC']-df1_C['R_dlPFC'],\
             'acc': df1_I['acc']-df1_C['acc'],\
             'conflict': df1_I['conflict']-df1_C['conflict'],\
             'adapt': df1_I['adapt']-df1_C['adapt'] }
    df_I_C=df_I_C.append(df1_I_C,ignore_index=True)
#---------------------------------##
df=df.replace({'group': '1'}, 0)
df=df.replace({'group': '2'}, 1)

df_I_C=df_I_C.replace({'group': '1'}, 0)
df_I_C=df_I_C.replace({'group': '2'}, 1)

df_I=df_I.replace({'group': '1'}, 0)
df_I=df_I.replace({'group': '2'}, 1)

df_C=df_C.replace({'group': '1'}, 0)
df_C=df_C.replace({'group': '2'}, 1)
#---------------------------------##
df_C=df_C.drop(labels=['trial'],axis=1)
df_I=df_I.drop(labels=['trial'],axis=1)
##---------------------------------##
## Parse DataFrames
##---------------------------------##
df_hcs=df.where(df['group']=='0')
df_hcs=df_hcs.dropna()
df_pts=df.where(df['group']=='1')
df_pts=df_pts.dropna()

df_hcs_I=df_I.where(df_I['group']=='0')
df_hcs_I=df_hcs_I.dropna()
df_pts_I=df_I.where(df_I['group']=='1')
df_pts_I=df_pts_I.dropna()

df_hcs_C=df_C.where(df_C['group']=='0')
df_hcs_C=df_hcs_C.dropna()
df_pts_C=df_C.where(df_C['group']=='1')
df_pts_C=df_pts_C.dropna()

8.68087 beta exlcuded for /Users/emilyhahn/projects/msit_modeling/LSS_AVG/hc005.dACC_LSS_avg_file.1D
4.02871 beta exlcuded for /Users/emilyhahn/projects/msit_modeling/LSS_AVG/hc005.R_IFG_LSS_avg_file.1D
8.32747 beta exlcuded for /Users/emilyhahn/projects/msit_modeling/LSS_AVG/hc038.L_dlPFC_LSS_avg_file.1D
4.06931 beta exlcuded for /Users/emilyhahn/projects/msit_modeling/LSS_AVG/hc038.L_dlPFC_LSS_avg_file.1D
4.60933 beta exlcuded for /Users/emilyhahn/projects/msit_modeling/LSS_AVG/pp002.L_IFG_LSS_avg_file.1D
5.37477 beta exlcuded for /Users/emilyhahn/projects/msit_modeling/LSS_AVG/pp002.L_IFG_LSS_avg_file.1D


In [8]:
def OLS(equation,df_type,df_name):
    t_model = smf.ols(formula = '%s' % equation, data=df_type)
    t_lin_reg = t_model.fit()
    print 'EQUATION: %s' % equation
    print t_lin_reg.summary()
    return
##---------------------------------##
dACC_pred_rt_ROIs_group_cond='dACC ~ conflict + group + group*conflict'
# OLS(dACC_pred_rt_ROIs_group_cond,df,'ALL SUBJS | ALL DATA | %s' % dACC_pred_rt_ROIs_group_cond)

### Test an infinite amount of interactions

In [9]:
# rt_pred_ROIs_group_cond='rt ~ cond + group + dACC + L_IFG + R_IFG + L_dlPFC + R_dlPFC'
# dACC_pred_rt_ROIs_group_cond='dACC ~ cond + group + rt + L_IFG + R_IFG + L_dlPFC + R_dlPFC'
# L_IFG_pred_rt_ROIs_group_cond='L_IFG ~ cond + group + rt + dACC + R_IFG + L_dlPFC + R_dlPFC'
# R_IFG_pred_rt_ROIs_group_cond='R_IFG ~ cond + group + rt + L_IFG + dACC + L_dlPFC + R_dlPFC'
# L_dlPFC_pred_rt_ROIs_group_cond='L_dlPFC ~ cond + group + rt + L_IFG + R_IFG + dACC + R_dlPFC'
# R_dlPFC_pred_rt_ROIs_group_cond='R_dlPFC ~ cond + group + rt + L_IFG + R_IFG + L_dlPFC + dACC'

# OLS(rt_pred_ROIs_group_cond,df,'ALL SUBJS | ALL DATA | %s' % rt_pred_ROIs_group_cond)
# OLS(dACC_pred_rt_ROIs_group_cond,df,'ALL SUBJS | ALL DATA | %s' % dACC_pred_rt_ROIs_group_cond)
# OLS(L_IFG_pred_rt_ROIs_group_cond,df,'ALL SUBJS | ALL DATA | %s' % L_IFG_pred_rt_ROIs_group_cond)
# OLS(R_IFG_pred_rt_ROIs_group_cond,df,'ALL SUBJS | ALL DATA | %s' % R_IFG_pred_rt_ROIs_group_cond)
# OLS(L_dlPFC_pred_rt_ROIs_group_cond,df,'ALL SUBJS | ALL DATA | %s' % L_dlPFC_pred_rt_ROIs_group_cond)
# OLS(R_dlPFC_pred_rt_ROIs_group_cond,df,'ALL SUBJS | ALL DATA | %s' % R_dlPFC_pred_rt_ROIs_group_cond)

# ##---------------------------------##
# rt_pred_ROIs_cond='rt ~ cond + dACC + L_IFG + R_IFG + L_dlPFC + R_dlPFC'
# dACC_pred_rt_ROIs_cond='dACC ~ cond + rt + L_IFG + R_IFG + L_dlPFC + R_dlPFC'
# L_IFG_pred_rt_ROIs_cond='L_IFG ~ cond + rt + dACC + R_IFG + L_dlPFC + R_dlPFC'
# R_IFG_pred_rt_ROIs_cond='R_IFG ~ cond + rt + L_IFG + dACC + L_dlPFC + R_dlPFC'
# L_dlPFC_pred_rt_ROIs_cond='L_dlPFC ~ cond + rt + L_IFG + R_IFG + dACC + R_dlPFC'
# R_dlPFC_pred_rt_ROIs_cond='R_dlPFC ~ cond + rt + L_IFG + R_IFG + L_dlPFC + dACC'

# OLS(rt_pred_ROIs_cond,df,'ALL SUBJS | ALL DATA | %s' % rt_pred_ROIs_cond)
# OLS(dACC_pred_rt_ROIs_cond,df,'ALL SUBJS | ALL DATA | %s' % dACC_pred_rt_ROIs_cond)
# OLS(L_IFG_pred_rt_ROIs_cond,df,'ALL SUBJS | ALL DATA | %s' % L_IFG_pred_rt_ROIs_cond)
# OLS(R_IFG_pred_rt_ROIs_cond,df,'ALL SUBJS | ALL DATA | %s' % R_IFG_pred_rt_ROIs_cond)
# OLS(L_dlPFC_pred_rt_ROIs_cond,df,'ALL SUBJS | ALL DATA | %s' % L_dlPFC_pred_rt_ROIs_cond)
# OLS(R_dlPFC_pred_rt_ROIs_cond,df,'ALL SUBJS | ALL DATA | %s' % R_dlPFC_pred_rt_ROIs_cond)

# OLS(rt_pred_ROIs_cond,df_hcs,'CTRLS | ALL DATA | %s' % rt_pred_ROIs_cond)
# OLS(dACC_pred_rt_ROIs_cond,df_hcs,'CTRLS | ALL DATA | %s' % dACC_pred_rt_ROIs_cond)
# OLS(L_IFG_pred_rt_ROIs_cond,df_hcs,'CTRLS | ALL DATA | %s' % L_IFG_pred_rt_ROIs_cond)
# OLS(R_IFG_pred_rt_ROIs_cond,df_hcs,'CTRLS | ALL DATA | %s' % R_IFG_pred_rt_ROIs_cond)
# OLS(L_dlPFC_pred_rt_ROIs_cond,df_hcs,'CTRLS | ALL DATA | %s' % L_dlPFC_pred_rt_ROIs_cond)
# OLS(R_dlPFC_pred_rt_ROIs_cond,df_hcs,'CTRLS | ALL DATA | %s' % R_dlPFC_pred_rt_ROIs_cond)

# OLS(rt_pred_ROIs_cond,df_pts,'PTS | ALL DATA | %s' % rt_pred_ROIs_cond)
# OLS(dACC_pred_rt_ROIs_cond,df_pts,'PTS | ALL DATA | %s' % dACC_pred_rt_ROIs_cond)
# OLS(L_IFG_pred_rt_ROIs_cond,df_pts,'PTS | ALL DATA | %s' % L_IFG_pred_rt_ROIs_cond)
# OLS(R_IFG_pred_rt_ROIs_cond,df_pts,'PTS | ALL DATA | %s' % R_IFG_pred_rt_ROIs_cond)
# OLS(L_dlPFC_pred_rt_ROIs_cond,df_pts,'PTS | ALL DATA | %s' % L_dlPFC_pred_rt_ROIs_cond)
# OLS(R_dlPFC_pred_rt_ROIs_cond,df_pts,'PTS | ALL DATA | %s' % R_dlPFC_pred_rt_ROIs_cond)
# ##---------------------------------##
# rt_pred_ROIs_group='rt ~ group + dACC + L_IFG + R_IFG + L_dlPFC + R_dlPFC'
# dACC_pred_rt_ROIs_group='dACC ~ group + rt + L_IFG + R_IFG + L_dlPFC + R_dlPFC'
# L_IFG_pred_rt_ROIs_group='L_IFG ~ group + rt + dACC + R_IFG + L_dlPFC + R_dlPFC'
# R_IFG_pred_rt_ROIs_group='R_IFG ~ group + rt + L_IFG + dACC + L_dlPFC + R_dlPFC'
# L_dlPFC_pred_rt_ROIs_group='L_dlPFC ~ group + rt + L_IFG + R_IFG + dACC + R_dlPFC'
# R_dlPFC_pred_rt_ROIs_group='R_dlPFC ~ group + rt + L_IFG + R_IFG + L_dlPFC + dACC'

# OLS(rt_pred_ROIs_group,df,'ALL SUBJS | ALL DATA | %s' % rt_pred_ROIs_group)
# OLS(dACC_pred_rt_ROIs_group,df,'ALL SUBJS | ALL DATA | %s' % dACC_pred_rt_ROIs_group)
# OLS(L_IFG_pred_rt_ROIs_group,df,'ALL SUBJS | ALL DATA | %s' % L_IFG_pred_rt_ROIs_group)
# OLS(R_IFG_pred_rt_ROIs_group,df,'ALL SUBJS | ALL DATA | %s' % R_IFG_pred_rt_ROIs_group)
# OLS(L_dlPFC_pred_rt_ROIs_group,df,'ALL SUBJS | ALL DATA | %s' % L_dlPFC_pred_rt_ROIs_group)
# OLS(R_dlPFC_pred_rt_ROIs_group,df,'ALL SUBJS | ALL DATA | %s' % R_dlPFC_pred_rt_ROIs_group)

# OLS(rt_pred_ROIs_group,df_I,'ALL SUBJS | MEAN INCONGRUENT TRIALS | %s' % rt_pred_ROIs_group)
# OLS(dACC_pred_rt_ROIs_group,df_I,'ALL SUBJS | MEAN INCONGRUENT TRIALS | %s' % dACC_pred_rt_ROIs_group)
# OLS(L_IFG_pred_rt_ROIs_group,df_I,'ALL SUBJS | MEAN INCONGRUENT TRIALS | %s' % L_IFG_pred_rt_ROIs_group)
# OLS(R_IFG_pred_rt_ROIs_group,df_I,'ALL SUBJS | MEAN INCONGRUENT TRIALS | %s' % R_IFG_pred_rt_ROIs_group)
# OLS(L_dlPFC_pred_rt_ROIs_group,df_I,'ALL SUBJS | MEAN INCONGRUENT TRIALS | %s' % L_dlPFC_pred_rt_ROIs_group)
# OLS(R_dlPFC_pred_rt_ROIs_group,df_I,'ALL SUBJS | MEAN INCONGRUENT TRIALS | %s' % R_dlPFC_pred_rt_ROIs_group)

# OLS(rt_pred_ROIs_group,df_C,'ALL SUBJS | MEAN CONGRUENT TRIALS | %s' % rt_pred_ROIs_group)
# OLS(dACC_pred_rt_ROIs_group,df_C,'ALL SUBJS | MEAN CONGRUENT TRIALS | %s' % dACC_pred_rt_ROIs_group)
# OLS(L_IFG_pred_rt_ROIs_group,df_C,'ALL SUBJS | MEAN CONGRUENT TRIALS | %s' % L_IFG_pred_rt_ROIs_group)
# OLS(R_IFG_pred_rt_ROIs_group,df_C,'ALL SUBJS | MEAN CONGRUENT TRIALS | %s' % R_IFG_pred_rt_ROIs_group)
# OLS(L_dlPFC_pred_rt_ROIs_group,df_C,'ALL SUBJS | MEAN CONGRUENT TRIALS | %s' % L_dlPFC_pred_rt_ROIs_group)
# OLS(R_dlPFC_pred_rt_ROIs_group,df_C,'ALL SUBJS | MEAN CONGRUENT TRIALS | %s' % R_dlPFC_pred_rt_ROIs_group)
# ##---------------------------------##
# dACC_pred_ROIs_cond='dACC ~ cond + L_IFG + R_IFG + L_dlPFC + R_dlPFC'
# L_IFG_pred_ROIs_cond='L_IFG ~ cond + dACC + R_IFG + L_dlPFC + R_dlPFC'
# R_IFG_pred_ROIs_cond='R_IFG ~ cond + L_IFG + dACC + L_dlPFC + R_dlPFC'
# L_dlPFC_pred_ROIs_cond='L_dlPFC ~ cond + L_IFG + R_IFG + dACC + R_dlPFC'
# R_dlPFC_pred_ROIs_cond='R_dlPFC ~ cond + L_IFG + R_IFG + L_dlPFC + dACC'

# OLS(dACC_pred_ROIs_cond,df,'ALL SUBJS | ALL DATA | %s' % dACC_pred_ROIs_cond)
# OLS(L_IFG_pred_ROIs_cond,df,'ALL SUBJS | ALL DATA | %s' % L_IFG_pred_ROIs_cond)
# OLS(R_IFG_pred_ROIs_cond,df,'ALL SUBJS | ALL DATA | %s' % R_IFG_pred_ROIs_cond)
# OLS(L_dlPFC_pred_ROIs_cond,df,'ALL SUBJS | ALL DATA | %s' % L_dlPFC_pred_ROIs_cond)
# OLS(R_dlPFC_pred_ROIs_cond,df,'ALL SUBJS | ALL DATA | %s' % R_dlPFC_pred_ROIs_cond)

# OLS(dACC_pred_ROIs_cond,df_hcs,'CTRLS | ALL DATA | %s' % dACC_pred_ROIs_cond)
# OLS(L_IFG_pred_ROIs_cond,df_hcs,'CTRLS | ALL DATA | %s' % L_IFG_pred_ROIs_cond)
# OLS(R_IFG_pred_ROIs_cond,df_hcs,'CTRLS | ALL DATA | %s' % R_IFG_pred_ROIs_cond)
# OLS(L_dlPFC_pred_ROIs_cond,df_hcs,'CTRLS | ALL DATA | %s' % L_dlPFC_pred_ROIs_cond)
# OLS(R_dlPFC_pred_ROIs_cond,df_hcs,'CTRLS | ALL DATA | %s' % R_dlPFC_pred_ROIs_cond)

# OLS(dACC_pred_ROIs_cond,df_pts,'PTS | ALL DATA | %s' % dACC_pred_ROIs_cond)
# OLS(L_IFG_pred_ROIs_cond,df_pts,'PTS | ALL DATA | %s' % L_IFG_pred_ROIs_cond)
# OLS(R_IFG_pred_ROIs_cond,df_pts,'PTS | ALL DATA | %s' % R_IFG_pred_ROIs_cond)
# OLS(L_dlPFC_pred_ROIs_cond,df_pts,'PTS | ALL DATA | %s' % L_dlPFC_pred_ROIs_cond)
# OLS(R_dlPFC_pred_ROIs_cond,df_pts,'PTS | ALL DATA | %s' % R_dlPFC_pred_ROIs_cond)
# ##---------------------------------##
# dACC_pred_rt_ROIs_group='dACC ~ group + L_IFG + R_IFG + L_dlPFC + R_dlPFC'
# L_IFG_pred_ROIs_group='L_IFG ~ group + dACC + R_IFG + L_dlPFC + R_dlPFC'
# R_IFG_pred_ROIs_group='R_IFG ~ group + L_IFG + dACC + L_dlPFC + R_dlPFC'
# L_dlPFC_pred_ROIs_group='L_dlPFC ~ group + L_IFG + R_IFG + dACC + R_dlPFC'
# R_dlPFC_pred_ROIs_group='R_dlPFC ~ group + L_IFG + R_IFG + L_dlPFC + dACC'

# OLS(dACC_pred_rt_ROIs_group,df,'ALL SUBJS | ALL DATA | %s' % dACC_pred_rt_ROIs_group)
# OLS(L_IFG_pred_ROIs_group,df,'ALL SUBJS | ALL DATA | %s' % L_IFG_pred_ROIs_group)
# OLS(R_IFG_pred_ROIs_group,df,'ALL SUBJS | ALL DATA | %s' % R_IFG_pred_ROIs_group)
# OLS(L_dlPFC_pred_ROIs_group,df,'ALL SUBJS | ALL DATA | %s' % L_dlPFC_pred_ROIs_group)
# OLS(R_dlPFC_pred_ROIs_group,df,'ALL SUBJS | ALL DATA | %s' % R_dlPFC_pred_ROIs_group)

# OLS(dACC_pred_rt_ROIs_group,df_I,'ALL SUBJS | MEAN INCONGRUENT TRIALS | %s' % dACC_pred_rt_ROIs_group)
# OLS(L_IFG_pred_ROIs_group,df_I,'ALL SUBJS | MEAN INCONGRUENT TRIALS | %s' % L_IFG_pred_ROIs_group)
# OLS(R_IFG_pred_ROIs_group,df_I,'ALL SUBJS | MEAN INCONGRUENT TRIALS | %s' % R_IFG_pred_ROIs_group)
# OLS(L_dlPFC_pred_ROIs_group,df_I,'ALL SUBJS | MEAN INCONGRUENT TRIALS | %s' % L_dlPFC_pred_ROIs_group)
# OLS(R_dlPFC_pred_ROIs_group,df_I,'ALL SUBJS | MEAN INCONGRUENT TRIALS | %s' % R_dlPFC_pred_ROIs_group)

# OLS(dACC_pred_rt_ROIs_group,df_C,'ALL SUBJS | MEAN CONGRUENT TRIALS | %s' % dACC_pred_rt_ROIs_group)
# OLS(L_IFG_pred_ROIs_group,df_C,'ALL SUBJS | MEAN CONGRUENT TRIALS | %s' % L_IFG_pred_ROIs_group)
# OLS(R_IFG_pred_ROIs_group,df_C,'ALL SUBJS | MEAN CONGRUENT TRIALS | %s' % R_IFG_pred_ROIs_group)
# OLS(L_dlPFC_pred_ROIs_group,df_C,'ALL SUBJS | MEAN CONGRUENT TRIALS | %s' % L_dlPFC_pred_ROIs_group)
# OLS(R_dlPFC_pred_ROIs_group,df_C,'ALL SUBJS | MEAN CONGRUENT TRIALS | %s' % R_dlPFC_pred_ROIs_group)
# ##---------------------------------##
# rt_pred_ROIs='rt ~ dACC + L_IFG + R_IFG + L_dlPFC + R_dlPFC'
# dACC_pred_rt_ROIs='dACC ~ rt + L_IFG + R_IFG + L_dlPFC + R_dlPFC'
# L_IFG_pred_rt_ROIs='L_IFG ~ rt + dACC + R_IFG + L_dlPFC + R_dlPFC'
# R_IFG_pred_rt_ROIs='R_IFG ~ rt + L_IFG + dACC + L_dlPFC + R_dlPFC'
# L_dlPFC_pred_rt_ROIs='L_dlPFC ~ rt + L_IFG + R_IFG + dACC + R_dlPFC'
# R_dlPFC_pred_rt_ROIs='R_dlPFC ~ rt + L_IFG + R_IFG + L_dlPFC + dACC'

# OLS(rt_pred_ROIs,df,'ALL SUBJS | ALL DATA | %s' % rt_pred_ROIs)
# OLS(______,df,'ALL SUBJS | ALL DATA | %s' % ______)
# OLS(______,df,'ALL SUBJS | ALL DATA | %s' % ______)
# OLS(______,df,'ALL SUBJS | ALL DATA | %s' % ______)
# OLS(______,df,'ALL SUBJS | ALL DATA | %s' % ______)
# OLS(______,df,'ALL SUBJS | ALL DATA | %s' % ______)

# OLS(rt_pred_ROIs,df_hcs,'CTRLS | ALL DATA | %s' % rt_pred_ROIs)
# OLS(______,df_hcs,'CTRLS | ALL DATA | %s' % ______)
# OLS(______,df_hcs,'CTRLS | ALL DATA | %s' % ______)
# OLS(______,df_hcs,'CTRLS | ALL DATA | %s' % ______)
# OLS(______,df_hcs,'CTRLS | ALL DATA | %s' % ______)
# OLS(______,df_hcs,'CTRLS | ALL DATA | %s' % ______)

# OLS(rt_pred_ROIs,df_pts,'PTS | ALL DATA | %s' % rt_pred_ROIs)
# OLS(______,df_pts,'PTS | ALL DATA | %s' % ______)
# OLS(______,df_pts,'PTS | ALL DATA | %s' % ______)
# OLS(______,df_pts,'PTS | ALL DATA | %s' % ______)
# OLS(______,df_pts,'PTS | ALL DATA | %s' % ______)
# OLS(______,df_pts,'PTS | ALL DATA | %s' % ______)

# OLS(rt_pred_ROIs,df_I,'ALL SUBJS | MEAN INCONGRUENT TRIALS | %s' % rt_pred_ROIs)
# OLS(______,df_I,'ALL SUBJS | MEAN INCONGRUENT TRIALS | %s' % ______)
# OLS(______,df_I,'ALL SUBJS | MEAN INCONGRUENT TRIALS | %s' % ______)
# OLS(______,df_I,'ALL SUBJS | MEAN INCONGRUENT TRIALS | %s' % ______)
# OLS(______,df_I,'ALL SUBJS | MEAN INCONGRUENT TRIALS | %s' % ______)
# OLS(______,df_I,'ALL SUBJS | MEAN INCONGRUENT TRIALS | %s' % ______)

# OLS(rt_pred_ROIs,df_C,'ALL SUBJS | MEAN CONGRUENT TRIALS | %s' % rt_pred_ROIs)
# OLS(______,df_C,'ALL SUBJS | MEAN CONGRUENT TRIALS | %s' % ______)
# OLS(______,df_C,'ALL SUBJS | MEAN CONGRUENT TRIALS | %s' % ______)
# OLS(______,df_C,'ALL SUBJS | MEAN CONGRUENT TRIALS | %s' % ______)
# OLS(______,df_C,'ALL SUBJS | MEAN CONGRUENT TRIALS | %s' % ______)
# OLS(______,df_C,'ALL SUBJS | MEAN CONGRUENT TRIALS | %s' % ______)

# OLS(rt_pred_ROIs,df_hcs_I,'CTRLS | MEAN INCONGRUENT TRIALS | %s' % rt_pred_ROIs)
# OLS(______,df_hcs_I,'CTRLS | MEAN INCONGRUENT TRIALS | %s' % ______)
# OLS(______,df_hcs_I,'CTRLS | MEAN INCONGRUENT TRIALS | %s' % ______)
# OLS(______,df_hcs_I,'CTRLS | MEAN INCONGRUENT TRIALS | %s' % ______)
# OLS(______,df_hcs_I,'CTRLS | MEAN INCONGRUENT TRIALS | %s' % ______)
# OLS(______,df_hcs_I,'CTRLS | MEAN INCONGRUENT TRIALS | %s' % ______)

# OLS(rt_pred_ROIs,df_pts_I,'PTS | MEAN INCONGRUENT TRIALS | %s' % rt_pred_ROIs)
# OLS(______,df_pts_I,'PTS | MEAN INCONGRUENT TRIALS | %s' % ______)
# OLS(______,df_pts_I,'PTS | MEAN INCONGRUENT TRIALS | %s' % ______)
# OLS(______,df_pts_I,'PTS | MEAN INCONGRUENT TRIALS | %s' % ______)
# OLS(______,df_pts_I,'PTS | MEAN INCONGRUENT TRIALS | %s' % ______)
# OLS(______,df_pts_I,'PTS | MEAN INCONGRUENT TRIALS | %s' % ______)

# OLS(rt_pred_ROIs,df_hcs_C,'CTRLS | MEAN CONGRUENT TRIALS | %s' % rt_pred_ROIs)
# OLS(______,df_hcs_C,'CTRLS | MEAN CONGRUENT TRIALS | %s' % ______)
# OLS(______,df_hcs_C,'CTRLS | MEAN CONGRUENT TRIALS | %s' % ______)
# OLS(______,df_hcs_C,'CTRLS | MEAN CONGRUENT TRIALS | %s' % ______)
# OLS(______,df_hcs_C,'CTRLS | MEAN CONGRUENT TRIALS | %s' % ______)
# OLS(______,df_hcs_C,'CTRLS | MEAN CONGRUENT TRIALS | %s' % ______)

# OLS(rt_pred_ROIs,df_pts_C,'PTS | MEAN CONGRUENT TRIALS | %s' % rt_pred_ROIs)
# OLS(______,df_pts_C,'PTS | MEAN CONGRUENT TRIALS | %s' % ______)
# OLS(______,df_pts_C,'PTS | MEAN CONGRUENT TRIALS | %s' % ______)
# OLS(______,df_pts_C,'PTS | MEAN CONGRUENT TRIALS | %s' % ______)
# OLS(______,df_pts_C,'PTS | MEAN CONGRUENT TRIALS | %s' % ______)
# OLS(______,df_pts_C,'PTS | MEAN CONGRUENT TRIALS | %s' % ______)
# ##---------------------------------##
# dACC_pred_rt_ROIs_group_cond='dACC ~ cond + rt + group'
# L_IFG_pred_rt_ROIs_group_cond='L_IFG ~ cond + group + rt'
# R_IFG_pred_rt_ROIs_group_cond='R_IFG ~ cond + group + rt'
# L_dlPFC_pred_rt_ROIs_group_cond='L_dlPFC ~ cond + group + rt'
# R_dlPFC_pred_rt_ROIs_group_cond='R_dlPFC ~ cond + group + rt'

# OLS(dACC_pred_rt_ROIs_group_cond,df,'ALL SUBJS | ALL DATA | %s' % dACC_pred_rt_ROIs_group_cond)
# OLS(______,df,'ALL SUBJS | ALL DATA | %s' % ______)
# OLS(______,df,'ALL SUBJS | ALL DATA | %s' % ______)
# OLS(______,df,'ALL SUBJS | ALL DATA | %s' % ______)
# OLS(______,df,'ALL SUBJS | ALL DATA | %s' % ______)
# OLS(______,df,'ALL SUBJS | ALL DATA | %s' % ______)
# ##---------------------------------##
# rt_pred_group_cond='rt ~ cond + group'
# dACC_pred_group_cond='dACC ~ cond + group'
# L_IFG_pred_group_cond='L_IFG ~ cond + group'
# R_IFG_pred_group_cond='R_IFG ~ cond + group'
# L_dlPFC_pred_group_cond='L_dlPFC ~ cond + group'
# R_dlPFC_pred_group_cond='R_dlPFC ~ cond + group'

# OLS(______,df,'ALL SUBJS | ALL DATA | %s' % ______)
# OLS(______,df,'ALL SUBJS | ALL DATA | %s' % ______)
# OLS(______,df,'ALL SUBJS | ALL DATA | %s' % ______)
# OLS(______,df,'ALL SUBJS | ALL DATA | %s' % ______)
# OLS(______,df,'ALL SUBJS | ALL DATA | %s' % ______)
# OLS(______,df,'ALL SUBJS | ALL DATA | %s' % ______)
# ##---------------------------------##
# dACC_pred_rt='dACC ~ rt'
# L_IFG_pred_rt='L_IFG ~ rt'
# R_IFG_pred_rt='R_IFG ~ rt'
# L_dlPFC_pred_rt='L_dlPFC ~ rt'
# R_dlPFC_pred_rt='R_dlPFC ~ rt'

# OLS(______,df,'ALL SUBJS | ALL DATA | %s' % ______)
# OLS(______,df,'ALL SUBJS | ALL DATA | %s' % ______)
# OLS(______,df,'ALL SUBJS | ALL DATA | %s' % ______)
# OLS(______,df,'ALL SUBJS | ALL DATA | %s' % ______)
# OLS(______,df,'ALL SUBJS | ALL DATA | %s' % ______)
# OLS(______,df,'ALL SUBJS | ALL DATA | %s' % ______)

# OLS(______,df_hcs,'CTRLS | ALL DATA | %s' % ______)
# OLS(______,df_hcs,'CTRLS | ALL DATA | %s' % ______)
# OLS(______,df_hcs,'CTRLS | ALL DATA | %s' % ______)
# OLS(______,df_hcs,'CTRLS | ALL DATA | %s' % ______)
# OLS(______,df_hcs,'CTRLS | ALL DATA | %s' % ______)
# OLS(______,df_hcs,'CTRLS | ALL DATA | %s' % ______)

# OLS(______,df_pts,'PTS | ALL DATA | %s' % ______)
# OLS(______,df_pts,'PTS | ALL DATA | %s' % ______)
# OLS(______,df_pts,'PTS | ALL DATA | %s' % ______)
# OLS(______,df_pts,'PTS | ALL DATA | %s' % ______)
# OLS(______,df_pts,'PTS | ALL DATA | %s' % ______)
# OLS(______,df_pts,'PTS | ALL DATA | %s' % ______)

# OLS(______,df_I,'ALL SUBJS | MEAN INCONGRUENT TRIALS | %s' % ______)
# OLS(______,df_I,'ALL SUBJS | MEAN INCONGRUENT TRIALS | %s' % ______)
# OLS(______,df_I,'ALL SUBJS | MEAN INCONGRUENT TRIALS | %s' % ______)
# OLS(______,df_I,'ALL SUBJS | MEAN INCONGRUENT TRIALS | %s' % ______)
# OLS(______,df_I,'ALL SUBJS | MEAN INCONGRUENT TRIALS | %s' % ______)
# OLS(______,df_I,'ALL SUBJS | MEAN INCONGRUENT TRIALS | %s' % ______)

# OLS(______,df_C,'ALL SUBJS | MEAN CONGRUENT TRIALS | %s' % ______)
# OLS(______,df_C,'ALL SUBJS | MEAN CONGRUENT TRIALS | %s' % ______)
# OLS(______,df_C,'ALL SUBJS | MEAN CONGRUENT TRIALS | %s' % ______)
# OLS(______,df_C,'ALL SUBJS | MEAN CONGRUENT TRIALS | %s' % ______)
# OLS(______,df_C,'ALL SUBJS | MEAN CONGRUENT TRIALS | %s' % ______)
# OLS(______,df_C,'ALL SUBJS | MEAN CONGRUENT TRIALS | %s' % ______)

# OLS(______,df_hcs_I,'CTRLS | MEAN INCONGRUENT TRIALS | %s' % ______)
# OLS(______,df_hcs_I,'CTRLS | MEAN INCONGRUENT TRIALS | %s' % ______)
# OLS(______,df_hcs_I,'CTRLS | MEAN INCONGRUENT TRIALS | %s' % ______)
# OLS(______,df_hcs_I,'CTRLS | MEAN INCONGRUENT TRIALS | %s' % ______)
# OLS(______,df_hcs_I,'CTRLS | MEAN INCONGRUENT TRIALS | %s' % ______)
# OLS(______,df_hcs_I,'CTRLS | MEAN INCONGRUENT TRIALS | %s' % ______)

# OLS(______,df_pts_I,'PTS | MEAN INCONGRUENT TRIALS | %s' % ______)
# OLS(______,df_pts_I,'PTS | MEAN INCONGRUENT TRIALS | %s' % ______)
# OLS(______,df_pts_I,'PTS | MEAN INCONGRUENT TRIALS | %s' % ______)
# OLS(______,df_pts_I,'PTS | MEAN INCONGRUENT TRIALS | %s' % ______)
# OLS(______,df_pts_I,'PTS | MEAN INCONGRUENT TRIALS | %s' % ______)
# OLS(______,df_pts_I,'PTS | MEAN INCONGRUENT TRIALS | %s' % ______)

# OLS(______,df_hcs_C,'CTRLS | MEAN CONGRUENT TRIALS | %s' % ______)
# OLS(______,df_hcs_C,'CTRLS | MEAN CONGRUENT TRIALS | %s' % ______)
# OLS(______,df_hcs_C,'CTRLS | MEAN CONGRUENT TRIALS | %s' % ______)
# OLS(______,df_hcs_C,'CTRLS | MEAN CONGRUENT TRIALS | %s' % ______)
# OLS(______,df_hcs_C,'CTRLS | MEAN CONGRUENT TRIALS | %s' % ______)
# OLS(______,df_hcs_C,'CTRLS | MEAN CONGRUENT TRIALS | %s' % ______)

# OLS(______,df_pts_C,'PTS | MEAN CONGRUENT TRIALS | %s' % ______)
# OLS(______,df_pts_C,'PTS | MEAN CONGRUENT TRIALS | %s' % ______)
# OLS(______,df_pts_C,'PTS | MEAN CONGRUENT TRIALS | %s' % ______)
# OLS(______,df_pts_C,'PTS | MEAN CONGRUENT TRIALS | %s' % ______)
# OLS(______,df_pts_C,'PTS | MEAN CONGRUENT TRIALS | %s' % ______)
# OLS(______,df_pts_C,'PTS | MEAN CONGRUENT TRIALS | %s' % ______)
# ##---------------------------------##
# rt_pred_cond='rt ~ cond'
# dACC_pred_cond='dACC ~ cond'
# L_IFG_pred_cond='L_IFG ~ cond'
# R_IFG_pred_cond='R_IFG ~ cond'
# L_dlPFC_pred_cond='L_dlPFC ~ cond'
# R_dlPFC_pred_cond='R_dlPFC ~ cond'

# OLS(______,df,'ALL SUBJS | ALL DATA')
# OLS(______,df,'ALL SUBJS | ALL DATA')
# OLS(______,df,'ALL SUBJS | ALL DATA')
# OLS(______,df,'ALL SUBJS | ALL DATA')
# OLS(______,df,'ALL SUBJS | ALL DATA')
# OLS(______,df,'ALL SUBJS | ALL DATA')

# OLS(______,df_hcs,'CTRLS | ALL DATA')
# OLS(______,df_hcs,'CTRLS | ALL DATA')
# OLS(______,df_hcs,'CTRLS | ALL DATA')
# OLS(______,df_hcs,'CTRLS | ALL DATA')
# OLS(______,df_hcs,'CTRLS | ALL DATA')
# OLS(______,df_hcs,'CTRLS | ALL DATA')

# OLS(______,df_pts,'PTS | ALL DATA')
# OLS(______,df_pts,'PTS | ALL DATA')
# OLS(______,df_pts,'PTS | ALL DATA')
# OLS(______,df_pts,'PTS | ALL DATA')
# OLS(______,df_pts,'PTS | ALL DATA')
# OLS(______,df_pts,'PTS | ALL DATA')
# ##---------------------------------##
# rt_pred_group='rt ~ group'
# dACC_pred_group='dACC ~ group'
# L_IFG_pred_group='L_IFG ~ group'
# R_IFG_pred_group='R_IFG ~ group'
# L_dlPFC_pred_group='L_dlPFC ~ group'
# R_dlPFC_pred_group='R_dlPFC ~ group'

# OLS(______,df,'ALL SUBJS | ALL DATA')
# OLS(______,df,'ALL SUBJS | ALL DATA')
# OLS(______,df,'ALL SUBJS | ALL DATA')
# OLS(______,df,'ALL SUBJS | ALL DATA')
# OLS(______,df,'ALL SUBJS | ALL DATA')
# OLS(______,df,'ALL SUBJS | ALL DATA')

# OLS(______,df_I,'ALL SUBJS | MEAN INCONGRUENT TRIALS')
# OLS(______,df_I,'ALL SUBJS | MEAN INCONGRUENT TRIALS')
# OLS(______,df_I,'ALL SUBJS | MEAN INCONGRUENT TRIALS')
# OLS(______,df_I,'ALL SUBJS | MEAN INCONGRUENT TRIALS')
# OLS(______,df_I,'ALL SUBJS | MEAN INCONGRUENT TRIALS')
# OLS(______,df_I,'ALL SUBJS | MEAN INCONGRUENT TRIALS')

# OLS(______,df_C,'ALL SUBJS | MEAN CONGRUENT TRIALS')
# OLS(______,df_C,'ALL SUBJS | MEAN CONGRUENT TRIALS')
# OLS(______,df_C,'ALL SUBJS | MEAN CONGRUENT TRIALS')
# OLS(______,df_C,'ALL SUBJS | MEAN CONGRUENT TRIALS')
# OLS(______,df_C,'ALL SUBJS | MEAN CONGRUENT TRIALS')
# OLS(______,df_C,'ALL SUBJS | MEAN CONGRUENT TRIALS')
# ##---------------------------------##